In [ ]:
#This will be the data analytics code

In [1]:
#Run if not downloaded already
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install matplotlib

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.6/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.6/libexec/bin/python3.8 -m pip install --upgrade pip' command.
     |████████████████████████████████| 7.2 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 29.0 MB 55 kB/s s eta 0:00:01
     |████████████████████████████████| 301 kB 24.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.6/libexec/bin/python3.8 -m pip install --upgrade pip' command.


In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial, reduce 
from sklearn.model_selection import train_test_split
# For either K Nearest Neighbor or K Mean Clustering, I don't know which to use yet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

#Load all the data into seperate data frames

# GDP
gdp_df = pd.read_csv("../clean_gdp_capita.csv", encoding="ISO-8859-1", index_col=0)
a = gdp_df.columns[3:]
gdp_df = pd.melt(gdp_df, id_vars=['Country.Name', 'Region', 'IncomeGroup'], value_vars=a,
        var_name='Year', value_name='GDP').sort_values(['Country.Name','Year']).reset_index()
gdp_df.drop('index', inplace=True, axis=1)


# GINI_Index
gini_df = pd.read_csv("../clean_gini_index.csv", encoding="ISO-8859-1", index_col=0)
a = gini_df.columns[3:]
gini_df = pd.melt(gini_df, id_vars=['Country.Name', 'Region', 'IncomeGroup'], value_vars=a,
        var_name='Year', value_name='Gini Index').sort_values(['Country.Name','Year']).reset_index()
gini_df.drop('index', inplace=True, axis=1)

# Literacy Rate
literacy_df = pd.read_csv("../clean_literacy_rate.csv", encoding="ISO-8859-1", index_col=0)
a = literacy_df.columns[3:]
literacy_df = pd.melt(literacy_df, id_vars=['Country.Name', 'Region', 'IncomeGroup'], value_vars=a,
        var_name='Year', value_name='Literacy Rate').sort_values(['Country.Name','Year']).reset_index()
literacy_df.drop('index', inplace=True, axis=1)

# Poverty Head
poverty_df = pd.read_csv("../clean_poverty_head.csv", encoding="ISO-8859-1", index_col=0)
a = poverty_df.columns[3:]
poverty_df = pd.melt(poverty_df, id_vars=['Country.Name', 'Region', 'IncomeGroup'], value_vars=a,
        var_name='Year', value_name='Poverty Rate').sort_values(['Country.Name','Year']).reset_index()
poverty_df.drop('index', inplace=True, axis=1)

# Secondary School Enrollment 
second_school_df = pd.read_csv("../clean_school_enrol_sec.csv", encoding="ISO-8859-1", index_col=0)
a = second_school_df.columns[3:]
second_school_df = pd.melt(second_school_df, id_vars=['Country.Name', 'Region', 'IncomeGroup'], value_vars=a,
        var_name='Year', value_name='Secondary School Enrollment').sort_values(['Country.Name','Year']).reset_index()
second_school_df.drop('index', inplace=True, axis=1)

# Tertiary School Enrollment 
third_school_df = pd.read_csv("../clean_school_enrol_tert.csv", encoding="ISO-8859-1", index_col=0)
a = third_school_df.columns[3:]
third_school_df = pd.melt(third_school_df, id_vars=['Country.Name', 'Region', 'IncomeGroup'], value_vars=a,
        var_name='Year', value_name='Tertinary School Enrollment').sort_values(['Country.Name','Year']).reset_index()
third_school_df.drop('index', inplace=True, axis=1)

In [89]:
# Merge dataframes
dfs = [gdp_df, gini_df, literacy_df, second_school_df, third_school_df, poverty_df]
final_df = reduce(lambda x,y: pd.merge(x,y, on=['Country.Name', 'Region', 'IncomeGroup', 'Year'], how='outer'), dfs)


In [90]:
final_df.dropna(subset=['Poverty Rate','GDP','Gini Index', 'Literacy Rate', 'Secondary School Enrollment'
                        ,'Tertinary School Enrollment'], inplace=True)
final_df.index.name = 'Index'
final_df

,Country.Name,Region,IncomeGroup,Year,GDP,Gini Index,Literacy Rate,Secondary School Enrollment,Tertinary School Enrollment,Poverty Rate
Index,,,,,,,,,,
61,Albania,Europe & Central Asia,Upper middle income,1960,639.484736,27.000000,98.712982,84.974258,13.90814,0.900000
62,Albania,Europe & Central Asia,Upper middle income,1961,639.484736,27.000000,98.712982,84.974258,13.90814,0.900000
63,Albania,Europe & Central Asia,Upper middle income,1962,639.484736,27.000000,98.712982,84.974258,13.90814,0.900000
64,Albania,Europe & Central Asia,Upper middle income,1963,639.484736,27.000000,98.712982,84.974258,13.90814,0.900000
65,Albania,Europe & Central Asia,Upper middle income,1964,639.484736,27.000000,98.712982,84.974258,13.90814,0.900000
...,...,...,...,...,...,...,...,...,...,...
16038,Zimbabwe,Sub-Saharan Africa,Lower middle income,2016,1464.583529,44.116667,88.693420,52.405670,10.00655,31.816667
16039,Zimbabwe,Sub-Saharan Africa,Lower middle income,2017,1548.170056,44.300000,88.693420,52.405670,10.00655,33.900000
16040,Zimbabwe,Sub-Saharan Africa,Lower middle income,2018,1683.740577,44.300000,88.693420,52.405670,10.00655,33.900000


In [91]:
y = final_df['Poverty Rate']
final_df.drop(['Poverty Rate'], axis=1, inplace = True)
X = final_df.select_dtypes(exclude=['object'])

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = .75, test_size = 0.25 , random_state = 0)

Index
8819      0.000000
6566     51.200000
14328     4.083333
7248      0.000000
1421     17.800000
Name: Poverty Rate, dtype: float64